# Basic Pyo Synth with 2 Oscillators

In [1]:
from pyo import *
from tkinter import *
import ipywidgets as wg
from ipywidgets import HBox, VBox
from IPython.display import display
import numpy as np
import cv2
import time




WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter (if available).
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



Helper functions

In [2]:
def mix(ratio): #amount in percentage used for oscillator one
    return 1.0 - ratio    

In [3]:
#https://gist.github.com/YuxiUx/ef84328d95b10d0fcbf537de77b936cd
def noteToFreq(note):
    a = 440 #frequency of A (coomon value is 440Hz)
    return (a / 32) * (2 ** ((note - 9) / 12))


In [4]:
def note_events(status, data1, data2): #Add detune parameter to osc_b then freq+detune for b freq
    
    if status == 144: #Note on
        
        #print(status, data1, data2)
        
        freq = noteToFreq(data1)
        #print(data1)
        amp = data2/128
        #print(amp)
        oscA_L.setFreq(freq)
        oscA_R.setFreq(freq)
        oscB_L.setFreq(freq)
        oscB_R.setFreq(freq)
        
        oscA_L.setMul(amp)
        oscA_R.setMul(amp)
        oscB_L.setMul(amp)
        oscB_R.setMul(amp)  
        
    if status == 128: #Note off
        
        #print(status, data1, data2)
        amp = 0
        oscA_L.setMul(amp)
        oscA_R.setMul(amp)
        oscB_L.setMul(amp)
        oscB_R.setMul(amp)

    
    #can also add other midi status cases, like Pitchbend, After touch, and Polyphony
    


In [5]:
def sel_waveform(left, right):
    
    tri = TriangleTable()
    sqr = SquareTable()
    saw = SawTable()
    sin = HarmTable([1])
    
    output = wg.Output()


    waveforms = wg.ToggleButtons(
        options=['Sine', 'Triangle', 'Sawtooth', 'Square'],
        )
    waveforms
    
    select_wave = wg.Button(
        description = 'Select')
    select_wave

    def set_wave(b):
        waveform = waveforms.value
        if waveform == 'Triangle':
            left.setTable(tri)
            right.setTable(tri)
            
        if waveform == 'Sine':
            left.setTable(sin)
            right.setTable(sin)
        if waveform == 'Sawtooth':
            left.setTable(saw)
            right.setTable(saw)
        if waveform == 'Square':
            left.setTable(sqr)
            right.setTable(sqr)   
        
        print(waveform, "wave selected. ")


    select_wave.on_click(set_wave)


    tab1 = wg.HBox([wg.VBox([waveforms, select_wave])])
    display(tab1, output)

    
    

In [6]:
def osc_controls(a_L, a_R, b_L, b_R):
    
    osc = wg.ToggleButtons(
        options=['Osc A', 'Osc B', 'Mix', 'Back'],)
    osc
    
    select_wave = wg.Button(
        description = 'Select')
    select_wave
    
    def selection(b):
        sel = osc.value
        if sel == 'Osc A':
            left = a_L
            right = a_R
        if sel == 'Osc B':
            left = b_L
            right = b_R
            
        if sel == 'Osc A' or sel == 'Osc B':
            wav_freq(sel, left, right)
        
          
        if sel == 'Mix': #move to different part
            print("Enter ratio of OscA to OscB (between 1.0-0):")
            ratio = float(input())
            a_L.setMul(ratio)
            a_R.setMul(ratio)
            b_L.setMul(mix(ratio))
            b_R.setMul(mix(ratio))
            
        if sel == 'Back':
            display_menu()
    

            
    select_wave.on_click(selection)
    tab1 = wg.HBox([wg.VBox([osc, select_wave])])
    display(tab1, output)
            
            
def wav_freq(sel, left, right):
            
    osc = wg.ToggleButtons(
    options=['Waveforms', 'Frequency', 'Back'],
    )
    osc

    select_wave = wg.Button(
    description = 'Select')
    select_wave
    def selection(b):
        sel = osc.value
        if sel == 'Waveforms':
            #select waveform
            sel_waveform(left, right)
        if sel == 'Frequency':
            
            def on_value_change(change): #for slider
                with output:
                    freq = change.new 
                    left.setFreq(freq)
                    right.setFreq(freq)
                
            f_slider = wg.IntSlider(value = 440, min=0, max = 2000)
            display(f_slider, output)
            f_slider.observe(on_value_change, names = 'value')
        

            
        if sel == 'Back':
            display_menu()
            
    select_wave.on_click(selection)
    tab1 = wg.HBox([wg.VBox([osc, select_wave])])
    display(tab1, output)
    

            
        
        

In [7]:
def filter_controls(a_L, a_R, b_L, b_R):
    
    print("0-Low pass, 1-Highpass")
    sel = int(input())
    print("Enter Cut off Frequency:")
    f = float(input())
    
    f1 = Biquad(a_L, freq=f, q=5, type=sel).out()
    f2 = Biquad(a_R, freq=f, q=5, type=sel).out(1)
    f3 = Biquad(b_L, freq=f, q=5, type=sel).out()
    f4 = Biquad(b_R, freq=f, q=5, type=sel).out(1)

    

# Main Program (Text Based)

In [9]:
#Main Program
s = Server()
s.setMidiInputDevice(99) # opens all devices
s.boot()
s.start()
output = wg.Output()


#Waveforms
tri = TriangleTable()
sqr = SquareTable()
saw = SawTable()
sin = HarmTable([1])

#Initial Values
amp = 0.3
freq = 220

#Oscillators
oscA_L = Osc(table=sin, freq = freq, mul = amp)
oscA_R = Osc(table=sin, freq = freq, mul = amp)
oscB_L = Osc(table=tri, freq = freq, mul = amp)
oscB_R = Osc(table=tri, freq = freq, mul = amp)

filterA_L = Biquad(oscA_L, freq=20000, q=3).out()
filterA_R = Biquad(oscA_R, freq=20000, q=3).out(1)
filterB_L = Biquad(oscB_L, freq=20000, q=3).out()
filterB_R = Biquad(oscB_R, freq=20000, q=3).out(1)

wait = 1

notes = RawMidi(note_events)
    
def display_menu():
    menu = wg.ToggleButtons(
    options=['Oscillators', 'Envelope', 'Filter', 'Amplitude','Object-Tracker', 'Quit'],
    description= 'MENU:',)
    menu

    select_wave = wg.Button(
    description = 'Select')
    select_wave
    
    def selection(b):
        sel = menu.value
        if sel == 'Oscillators':
            osc_controls(oscA_L, oscA_R, oscB_L, oscB_R)   

        if sel == 'Quit':
            print('Quitting')
            oscA_L.stop()
            oscA_R.stop()
            oscB_L.stop()
            oscB_R.stop() 
            s.stop()
            
        # OBJECT TRACKING OPTIONS
        if sel == 'Object-Tracker':
            
            #select midi or theremin mode
            mode = wg.ToggleButtons(
            options=['Theremin Mode', 'MIDI Mode'],
            )
            mode

            select_wave = wg.Button(
            description = 'Select')
            select_wave
            
            def selection(b):
                global theremin
                global midi
                theremin = 0
                midi = 0
                
                sel = mode.value
                if sel == 'Theremin Mode':
                    theremin = 1
                if sel == 'MIDI Mode':
                    midi= 1
            
                def drawBox(frame,bbox):
                    global x_prev, y_prev
                    #global flag
                    x_temp = 0
                    y_temp = 0

                    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
                    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
                    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

                    x_prev = x
                    y_prev = y

                    return x,x_temp,y,y_temp



                # define a video capture object 
                vid = cv2.VideoCapture(0) 
                time.sleep(3)


                #bit slower, more accurate
                tracker = cv2.TrackerCSRT_create()


                #starts loop
                ret, frame = vid.read() 

                bbox = cv2.selectROI("Tracking",frame, False)
                tracker.init(frame,bbox)

                while(True): 

                    timer = cv2.getTickCount() 
                    # Capture the video frame 
                    ret, frame = vid.read() 

                    #searches for object in frame
                    ret, bbox = tracker.update(frame)

                    #if object is found in frame
                    if ret:
                        x,x_prv, y, y_prv = drawBox(frame,bbox)

                        #update parameter values 
                        #amp = float(update_amp(y,y_prv))
                        if theremin == 1:
                            oscA_L.setFreq(x)
                            oscA_R.setFreq(x)
                            oscB_L.setFreq(x)
                            oscB_R.setFreq(x)
                            
                            #amplitude is (y/max y value) so that it never surpasses 1.0
                            amp = y/717
                            oscA_L.setMul(amp)
                            oscA_R.setMul(amp)
                            oscB_L.setMul(amp)
                            oscB_R.setMul(amp)
                            
                            
                        #update synth stream 
                        if midi == 1:
                            
                            #using y * 26 so that range goes from 0Hz- ~20,000Hz
                            f = y * 26
                            filterA_L.setFreq(f)
                            filterA_R.setFreq(f)
                            filterB_L.setFreq(f)
                            filterB_R.setFreq(f)
                            
                            #x/1279 so that the ratio is between 0.0 and 1.0
                            #meaning left is osc_B and right is osc_A
                            ratio = x/1279
                            a_L.setMul(ratio)
                            a_R.setMul(ratio)
                            b_L.setMul(mix(ratio))
                            b_R.setMul(mix(ratio))
                            


                    #if object is not found in frame        
                    else:
                        cv2.putText(frame,"Lost",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

                    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
                    cv2.putText(frame,str(int(fps)),(75,50), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

                    # Display the resulting frame 
                    cv2.imshow('frame', frame) 

                    # press q to quit
                    if cv2.waitKey(1) & 0xFF == ord('q'): 
                            break


                s.stop()            

                # After the loop release the cap object 
                vid.release() 
                # Destroy all the windows 
                cv2.destroyAllWindows() 
                
            select_wave.on_click(selection)
            tab1 = wg.VBox([mode, select_wave])
            display(tab1, output)

    select_wave.on_click(selection)
    tab1 = wg.VBox([menu, select_wave])
    display(tab1, output)

display_menu()



Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.


Output()

Output()

In [16]:
s.stop() 

In [ ]:
'''
        
        if text == 2:
            print()
            #still have to create envelope, will be better if we have midi to trigger it

        if text == 3:

            print("0-Low pass, 1-Highpass")
            sel = int(input())

            print("Enter Cut off Frequency (Off = 20,000 for LP and 0 for HP):")
            f = float(input())

            filterA_L.setType(sel)
            filterA_R.setType(sel)
            filterB_L.setType(sel)
            filterB_R.setType(sel)

            filterA_L.setFreq(f)
            filterA_R.setFreq(f)
            filterB_L.setFreq(f)
            filterB_R.setFreq(f)






#Waveforms
tri = TriangleTable()
sqr = SquareTable()
saw = SawTable()
sin = HarmTable([1])

#Initial Values
amp = 0.3
freq = 220

#Oscillators
oscA_L = Osc(table=sin, freq = freq, mul = amp)
oscA_R = Osc(table=sin, freq = freq, mul = amp)
oscB_L = Osc(table=tri, freq = freq, mul = amp)
oscB_R = Osc(table=tri, freq = freq, mul = amp)


filterA_L = Biquad(oscA_L, freq=20000, q=3).out()
filterA_R = Biquad(oscA_R, freq=20000, q=3).out(1)
filterB_L = Biquad(oscB_L, freq=20000, q=3).out()
filterB_R = Biquad(oscB_R, freq=20000, q=3).out(1)


#text = 1

#Menu loop
while text != 0:
    
    print("1-Oscillators, 2-Envelope, 3-Filter, 4-Amplitude, 0-Quit ")
    text = int(input())
    
    
    if text == 1:
        osc_controls(oscA_L, oscA_R, oscB_L, oscB_R)
        
    if text == 2:
        print()
        #still have to create envelope, will be better if we have midi to trigger it
    
    if text == 3:

        print("0-Low pass, 1-Highpass")
        sel = int(input())
        
        print("Enter Cut off Frequency (Off = 20,000 for LP and 0 for HP):")
        f = float(input())
        
        filterA_L.setType(sel)
        filterA_R.setType(sel)
        filterB_L.setType(sel)
        filterB_R.setType(sel)
        
        filterA_L.setFreq(f)
        filterA_R.setFreq(f)
        filterB_L.setFreq(f)
        filterB_R.setFreq(f)
        
'''    
            


# Just some random tests using Pyo objects down below

In [ ]:
#Oscillator Tests

s = Server()
s.boot()
s.start()

square = SquareTable()
tri = TriangleTable()


freq = 220

oscL = Osc(table=square, freq = freq, mul = 0.5).out()
oscR = Osc(table=square, freq = freq, mul= 0.5).out(1)

text = 1

while text != 0:
    
    text = int(input())
    
    if text == 2:
        oscL.setTable(tri)
        oscR.setTable(tri)
        
    if text == 1:
        oscL.setTable(square)
        oscR.setTable(square)

        
    if text == 4:
        print("Enter frequency:")
        f = int(input())
        oscL.setFreq(f)
        oscR.setFreq(f)

oscL.stop()
oscR.stop()

In [ ]:
oscL.stop()
oscR.stop()

In [ ]:
ta = HarmTable([1,.3,.2])
tb = HarmTable([1])
f = Fader(fadein=.1).play()
a = OscBank(ta,100,spread=0,frndf=.25,frnda=.01,num=[10,10],fjit=True,mul=f*0.5).out()
b = OscBank(tb,250,spread=.25,slope=.8,arndf=4,arnda=1,num=[10,10],mul=f*0.4).out()
b.ctrl()

In [ ]:
a.setMul(0.5)

In [ ]:
a.setFrnda(.1)
a.setSpread(0)
a.setFreq(220)
a.setFrndf(100)

In [ ]:
a.stop()
b.stop()

In [ ]:
s.stop()

In [ ]:
s = Server().boot()
s.start()
a = Noise(mul=.7)
lfo = Sine(freq=[.2, .25], mul=1000, add=1500)
f = Biquad(a, freq=lfo, q=5, type=2).out()
f = Biquad(a, freq=lfo, q=5, type=2).out(1)

In [ ]:
f.setType(0)
f.setFreq(300)

In [ ]:
s = Server().boot()
s.start()

tab = SawTable()
sound = Osc(table=tab, freq=180, mul=0.2)


In [ ]:
f= Biquad(sound, freq=400, q=5).out()


In [ ]:
sound.setTable(sin)

In [ ]:
s.stop()